# Polymarket API Filter

For our prediction market aggregator we curate markets that are interesting on polymarket. This is because we are just at a demo level of maturity at this point and need to hold the users hand. 

This notebook provides the tools for searching and filtering interesting markets

In [46]:
import requests
import pandas as pd
from datetime import datetime
import json

In [47]:
class PolymarketAPI:
    def __init__(self, base_url="https://gamma-api.polymarket.com"):
        self.base_url = base_url

    def search_events(self, keywords=None, min_liquidity=0, closed=False, active=True):
        params = {
            "closed": closed,
            "active": active,
            "liquidity_num_min": min_liquidity
        }
        if keywords:
            params["title"] = keywords
        response = requests.get(f"{self.base_url}/events", params=params)
        return response.json()
    
    def format_event_data(self, event):
        data = {
            'event_id': event.get('id', ''),
            'title': event.get('title', ''),
            'description': event.get('description', ''),
            'liquidity': float(event.get('liquidity', 0)),
            'volume': float(event.get('volume', 0)),
            'volume24hr': float(event.get('volume24hr', 0)),
            'endDate': event.get('endDate', ''),
            'url': f"https://polymarket.com/event/{event.get('slug', '')}",
            'markets': []
        }
        
        if event.get('markets'):
            for market in event['markets']:
                market_data = {
                    'market_id': market.get('id', ''),
                    'question': market.get('question', ''),
                    'condition_id': market.get('conditionId', ''),
                    'yes_token_id': None,
                    'no_token_id': None
                }
                
                # Extract token IDs from clobTokenIds if available
                if market.get('clobTokenIds'):
                    try:
                        token_ids = eval(market['clobTokenIds'])
                        market_data['yes_token_id'] = token_ids[0]
                        market_data['no_token_id'] = token_ids[1]
                    except:
                        pass
                
                # Extract prices
                if market.get('outcomePrices'):
                    try:
                        prices = eval(market['outcomePrices'])
                        label = market.get('groupItemTitle', 'option')
                        data[f'{label}_yes'] = float(prices[0])
                        data[f'{label}_no'] = float(prices[1])
                    except:
                        continue
                
                data['markets'].append(market_data)
                        
        return data

    def analyze_events(self, events):
        formatted_events = [self.format_event_data(event) for event in events]
        df = pd.DataFrame(formatted_events)
        
        numeric_cols = ['liquidity', 'volume', 'volume24hr']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').round(2)
        
        price_cols = [col for col in df.columns if col.endswith('_yes') or col.endswith('_no')]
        for col in price_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').round(4)
            
        return df.sort_values('volume', ascending=False)

In [52]:
# Example usage
api = PolymarketAPI()

# Search for high liquidity election events
events = api.search_events(keywords="election", min_liquidity=100000)
df = api.analyze_events(events)

print("\nEvent summary:")
display(df[['event_id', 'title', 'liquidity', 'volume24hr']].head())

if not df.empty:
    first_event = df.iloc[0]
    print(f"\nMarket details for {first_event['event_id']} event:")
    for market in first_event['markets']:
        print(f"\nMarket ID: {market['market_id']}")
        # print(f"Question: {market['question']}")
        # print(f"Condition ID: {market['condition_id']}")
        # print(f"YES Token ID: {market['yes_token_id']}")
        # print(f"NO Token ID: {market['no_token_id']}")



Event summary:


,event_id,title,liquidity,volume24hr
18,10717,# of Republican Senate seats after Election?,945616.64,59166.92
15,10558,How many named storms during Atlantic Hurrican...,180861.98,91370.96
0,10019,Pennsylvania Senate Election Winner,230330.66,90331.32
8,10259,Will Ukraine join NATO in 2024?,146986.39,57091.40
9,10262,U.S. Recession in 2024?,143258.52,1080.97



Market details for 10717 event:

Market ID: 501694

Market ID: 501695

Market ID: 501696

Market ID: 501697

Market ID: 501698

Market ID: 501699

Market ID: 501700

Market ID: 501701
